# How to Build a Tree Tensor Network
This notebook is an example on how to build custom tree tensor networks (TTN). As a first step we need to import PyTreeNet.

In [1]:
import numpy as np
from pytreenet.core import TreeTensorNetwork, Node, LegSpecification
from pytreenet.random import crandn, random_tensor_node

A TTN will always be generated empty.

In [2]:
ttn = TreeTensorNetwork()

The first node to be added is the root node. This has to be created and added.

In [3]:
root_node = Node(identifier="root")
root_tensor = crandn((1,2,3,4))

# The data tensor and the node are linked while adding them to the TTN
print(f"Nodes of the TTN before adding the root: {ttn.nodes}")
ttn.add_root(root_node, root_tensor)
print(f"Nodes of the TTN after adding the root: {ttn.nodes}")

Nodes of the TTN before adding the root: {}
Nodes of the TTN after adding the root: {'root': <pytreenet.core.node.Node object at 0x7f9f493fd940>}


The root node is usually the start of recursions and other methods to work with the TTN. There are multiple ways to access nodes in general and the root node specifically:

In [4]:
root_id = ttn.root_id

# Using the general way
print(ttn.nodes[root_id], ttn.tensors[root_id].shape)

# Using the general way to get the tensor as well
root_n, root_t = ttn[root_id]
print(root_n, root_t.shape)

# Using the root property
root_n, root_t = ttn.root
print(root_n, root_t.shape)

Node root
Parent: None
Children: []
Open legs: [0, 1, 2, 3]
Shape: (1, 2, 3, 4)
 (1, 2, 3, 4)
Node root
Parent: None
Children: []
Open legs: [0, 1, 2, 3]
Shape: (1, 2, 3, 4)
 (1, 2, 3, 4)
Node root
Parent: None
Children: []
Open legs: [0, 1, 2, 3]
Shape: (1, 2, 3, 4)
 (1, 2, 3, 4)


Now we would like to add children to the root.

In [5]:
# Creating multiple nodes and random tensors
shapes = {"child0": (3,5,5,2), "child1": (2,4,4), "child2": (2,3,3)}
child_nodes = [random_tensor_node(identifier=ident,shape=shape)
               for ident, shape in shapes.items()]

# We can now add them using the built-in function
ttn.add_child_to_parent(child_nodes[0][0], child_nodes[0][1], 3, root_id, 1)

Note that the shape of the nodes in the TTN changes. This ensures the constant leg ordering convention `(parent_leg, children_legs, open_legs)`:

In [6]:
print(f"Root node shape: {ttn.root[0].shape} (before it was (1,2,3,4))")
child_shape = ttn.nodes["child0"].shape
print(f"Shape of the first child node: {child_shape} (before it was (3,5,5,2))")

Root node shape: (2, 1, 3, 4) (before it was (1,2,3,4))
Shape of the first child node: (2, 3, 5, 5) (before it was (3,5,5,2))


In [7]:
# Add the remaining children
ttn.add_child_to_parent(child_nodes[1][0], child_nodes[1][1], 2, root_id, 3)
ttn.add_child_to_parent(child_nodes[2][0], child_nodes[2][1], 1, root_id, 3)

We can contract, i.e. combine two nodes:

In [8]:
ttn.contract_nodes("child0","root")
print("Remaining Nodes:", list(ttn.nodes.keys()))
print("Shape of the new node:", ttn.nodes["child0contrroot"].shape)

Remaining Nodes: ['child1', 'child2', 'child0contrroot']
Shape of the new node: (4, 3, 3, 5, 5, 1)


In [9]:
print(ttn.nodes["child0contrroot"])

Node child0contrroot
Parent: None
Children: ['child1', 'child2']
Open legs: [2, 3, 4, 5]
Shape: (4, 3, 3, 5, 5, 1)



And we can of course split nodes into two. In that case we need to define which leg should go where:

In [10]:
# Using QR decomposition
cont_node = ttn.nodes["child0contrroot"]
nvirt_legs = cont_node.nvirt_legs()
q_legs = LegSpecification(None, ["child1"],[nvirt_legs, nvirt_legs+1],
                              node=cont_node)
r_legs = LegSpecification(None, ["child2"],[nvirt_legs+2, nvirt_legs+3],
                              node=cont_node)
r_legs.is_root = True
ttn.split_node_qr("child0contrroot", q_legs, r_legs,
                  "new_child0", "new_root")

print("Remaining Nodes:", list(ttn.nodes.keys()))
print("New Root:\n", ttn.nodes["new_root"])
print("New Child:\n", ttn.nodes["new_child0"])

Remaining Nodes: ['child1', 'child2', 'new_child0', 'new_root']
New Root:
 Node new_root
Parent: None
Children: ['new_child0', 'child2']
Open legs: [2, 3]
Shape: (15, 3, 5, 1)

New Child:
 Node new_child0
Parent: new_root
Children: ['child1']
Open legs: [2, 3]
Shape: (15, 4, 3, 5)



We can also bring our TTN into a canonical for. This means all nodes but the one defined as the orthogonality center can be interpreted as isometries.

In [11]:
ttn.canonical_form("new_root")
print("Orthogonality Center:", ttn.orthogonality_center_id)

Orthogonality Center: new_root


This center can easily be moved around:

In [12]:
ttn.move_orthogonalization_center("new_child0")
print("Orthogonality Center:", ttn.orthogonality_center_id)
ttn.move_orthogonalization_center("child1")
print("Orthogonality Center:", ttn.orthogonality_center_id)

Orthogonality Center: new_child0
Orthogonality Center: child1


We can also contract the complete TTN at any point. However, for anything but toy and debugging cases, this can quickly lead to too high memory requirements.

In [13]:
result, order = ttn.completely_contract_tree(to_copy=True)
print("The tensors were contracted in the order:", order)
print("The resulting shape is:", result.shape)
print(f"This would be", np.prod(result.shape), "elements")

The tensors were contracted in the order: ['new_root', 'new_child0', 'child1', 'child2']
The resulting shape is: (5, 1, 3, 5, 2, 4, 2, 3)
This would be 3600 elements
